# Standardized Data With Timestamps Suitable For Backtesting

### [Data Dictionary](https://www.calcbench.com/home/standardizedmetrics)
### [Documentation](http://calcbench.github.io/python_api_client/html/numeric-data.html#point-in-time)

In [ ]:
%pip install calcbench-api-client[Pandas,Backoff] tqdm

In [ ]:
from pathlib import Path
from tqdm import tqdm
import pandas as pd

import calcbench as cb
from calcbench.downloaders import iterate_and_save_pandas

## Simple Example
### Get face points only for three companies

In [ ]:
data = pd.DataFrame()
for ticker in tqdm(["MSFT", "AMZN", "XOM"]):
    company_data = cb.standardized(company_identifiers=[ticker], point_in_time=True)
    data = pd.concat([data, company_data])
data.to_csv("pit_sample.csv")

## Download and Save Historic Data For All Companies

In [ ]:
tickers = cb.tickers(entire_universe=True)

In [ ]:
face_output_file_name = Path.joinpath(Path.home(), "cb_pit_data.csv")

iterate_and_save_pandas(
    tickers,
    lambda ticker: cb.standardized(
        company_identifiers=[ticker],
        point_in_time=True
    ),
    face_output_file_name,
)

## Push Notification of New Data
### Example @ https://github.com/calcbench/notebooks/tree/master/filing_listener

